In [ ]:
import pickle
import torch 
import ast
from tqdm import tqdm
import json
from itertools import islice
from ResearchGraphDataset import * 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd
df = pd.read_json('df.json')
df = df[df['publication_year']>=2020]

with open('splits_New.pkl', 'rb') as f:
	splits = pickle.load(f)

dataset = ResearchGraphDataset(df,splits, max_authors = 2) 

with open("data.json", "r", encoding="utf-8") as f: # prompt_data_preprocess获取的节点特征文件
    data = json.load(f)

idx = pd.read_csv(r'pos_idx.csv') # 正样本id数据

with open("ds.json", "r", encoding="utf-8") as f: # 原始论文数据
    prob = json.load(f)

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:353: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Processing papers: 100%|██████████| 12209/12209 [00:02<00:00, 5694.67it/s]
Processing collaborations: 12209it [00:01, 11708.78it/s]


In [8]:
active_authors,author_id_to_idx,idx_to_author_id = dataset.generate_authorid2idx(2023)
all_pairs = list(data["dist"].keys())

In [ ]:
split=2023
splits = dataset.splits[split]
train_g = dataset._add_topological_features(splits['train'])
with open( f'train_samples_{split}_Original.pkl', 'rb') as f:
    best_sample = pickle.load(f)

In [ ]:
rank_list = []
for each in tqdm(prob['final_prob']):
    rank_list.append([prob['agent'][each], each, prob['final_prob'][each]])
dfrank = pd.DataFrame(data = rank_list, columns = ['agent','pairs','prob'])
dfrank.sort_values(by = ['agent','prob'], ascending = False, inplace = True)
df1= pd.DataFrame(dfrank['agent'].value_counts())

100%|██████████| 20908/20908 [00:00<00:00, 996019.17it/s]


In [14]:
total_r,total_p = 0,0
hits = 0
cnt=0
for src in tqdm(df1.index):
    labels = {}
    n = 0
    for src_nodes, dst_nodes, label in best_sample:
        if (int(src)== src_nodes.item()):
            if int(label.item())== 1:
                labels[(src_nodes.item(), dst_nodes.item())] = int(label.item())
        if (int(src) == dst_nodes.item()):
            if int(label.item())== 1:
                labels[( dst_nodes.item(), src_nodes.item())] = int(label.item())
                
    if len(labels)==0:
        cnt+=1
        continue
    length = int(df1[df1.index == src]['count'].item()*0.5) + 1
    prob_rk = [eval(each) for each in dfrank[dfrank['agent']==int(src)].iloc[:length]['pairs'].tolist()]
    #print(prob_rk,labels)
    #hit@k
    if any(p in labels.keys() for p in prob_rk): 
        hits += 1
    #recall@k
    for p in labels.keys():
        if p in prob_rk:
            n +=1
    #precision@k
    '''for p in prob_rk:
        if p in labels.keys():
            precision+=1'''
    total_r += n/len(labels)
    total_p += n/length

100%|██████████| 3599/3599 [18:55<00:00,  3.17it/s]


In [15]:
total_r/len(df1) ,total_p/len(df1), hits/len(df1)

(0.8531770288925054, 0.3878530915655993, 0.9030286190608502)